In [1]:
import pandas as pd
import numpy as np
import scipy.stats as sps
from statsmodels.sandbox.stats.multicomp import multipletests

In [2]:
data = pd.read_csv("kr-vs-kp.data", header=None)
data = data.iloc[:,:10]
np.unique(data.iloc[:,:10].values)

array(['f', 't'], dtype=object)

Как видим эл-ты выборки принимают всего 2 значения.

In [3]:
data = data.applymap(lambda c: 1 if c == 'f' else 0)
Xs = [data[i] for i in range(10)]
Xs[0].shape

(3196,)

Проверяем попарную независимость выборок при помощи критерия хи-квадрат. Параллельно проверяем условие применимости критерия, а именно $\frac{n_im_j}{N} \geq 5$.

In [4]:
p_values = []
for i in range(10):
    for j in range(i + 1, 10):
        observed = np.histogram2d(Xs[i], Xs[j], bins=2)[0]
        assert observed.sum(axis=0).min() * observed.sum(axis=1).min() / 200 >= 5
        p_value = sps.chi2_contingency(observed)[1]
        p_values.append(p_value)

Проверим совместную независимость выборок при помощи коэффициент конкордации Кенделла:

In [5]:
R = np.array(map(np.argsort, Xs)).transpose()
n, k = R.shape
W = 12. / k**2 / (n**3 - n) * np.sum((R.sum(axis=1) - k*(n+1)/2.)**2)
p_value = sps.chi2.cdf(k * (n - 1) * W, n - 1) - 1.
p_values.append(p_value)
print p_value

0.0


Проведём поправку на множественную проверку гипотез. Используем метод Холма, т.к. это мощнейшая из процедур, контролирующих FWER, если характер зависимости между статистиками неизвестен (в нашем случае статистики считаются по одной и той же выборке, поэтому они могут быть зависимы).

In [6]:
results = multipletests(p_values, method='holm')
print results[:2]

(array([False,  True,  True, False, False,  True,  True,  True, False,
       False, False,  True, False,  True,  True,  True,  True,  True,
       False, False,  True,  True, False,  True,  True,  True, False,
        True, False, False, False,  True,  True,  True, False,  True,
       False, False,  True,  True,  True,  True,  True,  True, False,  True], dtype=bool), array([  1.00000000e+000,   9.06142936e-020,   2.22813828e-002,
         5.49365379e-001,   8.26636184e-001,   1.06257283e-009,
         1.54993788e-010,   2.00837950e-002,   1.31859186e-001,
         1.00000000e+000,   2.05489040e-001,   3.84514795e-014,
         1.91572093e-001,   2.65417022e-030,   1.52100675e-014,
         2.54156122e-004,   4.67516501e-002,   2.22813828e-002,
         1.00000000e+000,   2.10903593e-001,   6.95982999e-015,
         1.22144980e-002,   1.00000000e+000,   5.20923524e-004,
         2.43845776e-012,   7.79502124e-026,   7.88440727e-001,
         2.21420252e-004,   1.00000000e+000,   1.000

Итак, контроллируя FWER на уровне 0.05, процедура отвергла многие гипотезы о попарной независимости, а также гипотезу о совместной независимости.